In [1]:
from pymatgen.core import Structure
from pymatgen.io.vasp.sets import MPStaticSet
import math
import numpy as np
import pandas as pd

In [2]:
def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier

In [3]:
def neighbouring_atoms(s: Structure, elem: str, site: int, neighbouring_site_no: int):
    q = ""
    a= []
    test = np.array(s.distance_matrix[site])
    test.sort()
    r = round_up(test[neighbouring_site_no], 2) #it gets the length value for the 6th largest bond made by that site for octahedra. 
    summ = 0
    if s.sites[site].specie.name == elem:
#    q = s.sites[site].specie
#    if q.name == elem:
    
        for j in range(neighbouring_site_no):
            summ += s.get_distance(site,s.get_all_neighbors(r)[site][j].index)
            a.append(s.get_all_neighbors(r)[site][j].index)
    return a    

In [4]:
def avg_bl(s: Structure, elem: str, neighbouring_site_no: int):
    q = ""
    errr = ""
    df = pd.DataFrame(columns=['Site','Average Bond Length','Error(if any)'])
    for i in range(len(s)):
        test = np.array(s.distance_matrix[i])
        test.sort()
        r = round_up(test[neighbouring_site_no], 2) 
        #it gets the length value for the nth largest bond made by that site for the geometry it has. 
        summ = 0
        q = s.sites[i].specie
        if q.name == elem:
            a = []
            for j in range(neighbouring_site_no):
                summ += s.get_distance(i,s.get_all_neighbors(r)[i][j].index)
                a.append(s.get_all_neighbors(r)[i][j].index)
            average = summ/neighbouring_site_no
            data = {'Site': int(i),'Average Bond Length': average}
            df = df.append(data,ignore_index=True)
            for i in a:
                if i < 96:
                    errr = "V-V distance taken instead of V-O"
                    df.at[i, 'Error(if any)'] = errr
    return df

In [5]:
def increase_decrease_calculation(df,position: int):
    df.to_dict()
    df2 = df['Average Bond Length']
    l1 = df2[position]
    l2 = df2[position+1]
    inc_dec = ((l1-l2)/l2)*100
    return inc_dec

In [6]:
def polaron_introduction(s: Structure, site: int, bl_inc_dec: int, neighbouring_site_no: int):
    strain = 1+(bl_inc_dec/100)
    test = []
    elem = s.sites[site].specie.name
    x = neighbouring_atoms(s,elem,site,neighbouring_site_no)
    p = 0
    q = 0
#    test = np.array(s.distance_matrix[site])
#    test.sort()
#    r = round_up(test[neighbouring_site_no], 2)
    
#    for i in range(neighbouring_site_no):
#        x.append(s.get_all_neighbors(r)[site][i].index)
    
    for i in x:
        p = ((s[i].frac_coords-s[site].frac_coords)*strain)+s[site].frac_coords
        q = s[i].frac_coords
        for j in range(len(p)):
            if p[j]>1:
                p[j] = (((q[j]-1)*strain)+1)
        s[i].frac_coords = p
    return s

In [7]:
def polaron_along_one_direction(s: Structure, site1: int, site2: int, bl_inc_dec: int):
    strain = 1+(bl_inc_dec/100)
    p = 0
    q = 0

    p = ((s[site2].frac_coords-s[site1].frac_coords)*strain)+s[site1].frac_coords
    q = s[site2].frac_coords
    for j in range(len(p)):
        if p[j]>1:
            p[j] = (((q[j]-1)*strain)+1)
    s[site2].frac_coords = p
    return s

In [8]:
def common_atoms(s: Structure, site1: int, site2: int, neighbouring_site_no: int):
    a = []
    b = []
    yes = 0
    same = []

    test = np.array(s.distance_matrix[site1])
    test.sort()
    r = round_up(test[neighbouring_site_no], 2)
    for i in range(neighbouring_site_no):
        a.append(s.get_all_neighbors(r)[site1][i].index)

    test2 = np.array(s.distance_matrix[site2])
    test2.sort()
    r2 = round_up(test2[neighbouring_site_no], 2)
    for j in range(6):
        b.append(s.get_all_neighbors(r2)[site2][j].index)
        
    for i in a:
        for j in b:
            if i ==j:
                yes += 1
                same.append(i)
                
    if yes == 0:
        message = "No common O atoms"
        return message
    else:
        return same